In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=25.0&md=05&md=06&md=07&md=08&md=09&md=10&ts=1&et=10&mb=0&mt=9999999&cn=20&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113"
html = requests.get(base_url)
soup = BeautifulSoup(html.content, "html.parser")

pagination = soup.find("div", class_="pagination pagination_set-nav")
last_page_link = pagination.find_all("a")[-2]
last_page_number = int(last_page_link.text)

data = []

for page_number in range(1, last_page_number + 1):
    page_url = f"{base_url}&page={page_number}"
    page_html = requests.get(page_url)
    page_soup = BeautifulSoup(page_html.content, "html.parser")

    mansions = page_soup.select("div.cassetteitem")
    for mansion in mansions:
        name = mansion.find("div", class_="cassetteitem_content-title").get_text(strip=True)
        address = mansion.find("li", class_="cassetteitem_detail-col1").get_text(strip=True)
        year = mansion.find("li", class_="cassetteitem_detail-col3").div.get_text(strip=True)
        
        rooms = mansion.select("tr.js-cassette_link")
        for room in rooms:
            floor = room.find("td", string=True).get_text(strip=True)
            rent = room.find("span", class_="cassetteitem_price--rent").get_text(strip=True)
            administration = room.find("span", class_="cassetteitem_price--administration").get_text(strip=True)
            deposit = room.find("span", class_="cassetteitem_price--deposit").get_text(strip=True)
            gratuity = room.find("span", class_="cassetteitem_price--gratuity").get_text(strip=True)
            madori = room.find("span", class_="cassetteitem_madori").get_text(strip=True)
            menseki = room.find("span", class_="cassetteitem_menseki").get_text(strip=True)

            data.append({
                '物件名': name,
                '住所': address,
                '築年数': year,
                '階': floor,
                '家賃': rent,
                '管理費': administration,
                '敷金': deposit,
                '礼金': gratuity,
                '間取り': madori,
                '専有面積': menseki,
            })

df = pd.DataFrame(data)

In [2]:
df

,物件名,住所,築年数,階,家賃,管理費,敷金,礼金,間取り,専有面積
0,シャーメゾンステージ秋葉原,東京都千代田区神田佐久間河岸,築18年,12階,15.2万円,7000円,15.2万円,15.2万円,2K,40.42m2
1,パークウェル御茶ノ水,東京都千代田区神田須田町１,築20年,6階,14万円,10000円,14万円,14万円,2K,36.88m2
2,kinomori代々木上原,東京都渋谷区上原２,築1年,4階,12.7万円,10000円,-,-,2K,23.7m2
3,kinomori代々木上原,東京都渋谷区上原２,築1年,2階,13.1万円,10000円,-,-,2K,24.08m2
4,kinomori代々木上原,東京都渋谷区上原２,築1年,3階,13.2万円,10000円,-,-,2K,24.08m2
...,...,...,...,...,...,...,...,...,...,...
1030,プラチナコート南麻布（プラチナコートミナミアザブ）,東京都港区南麻布１,築19年,3階,24.1万円,10000円,-,-,2LDK,53.62m2
1031,東京メトロ南北線 麻布十番駅 8階建 築19年,東京都港区南麻布１,築19年,3階,24.1万円,10000円,-,-,2LDK,53.62m2
1032,都営大江戸線 若松河田駅 6階建 築9年,東京都新宿区河田町,築9年,5階,23万円,10000円,23万円,23万円,2LDK,53.48m2
1033,ザ・パークハビオ早稲田,東京都新宿区早稲田町,築6年,9階,25万円,23000円,25万円,25万円,2LDK,51.25m2


In [ ]:
# スプレッドシート形式で出力する
df.to_excel('output3.xlsx', index=False)  # index=Falseでインデックスを出力しないようにしています